https://data-flair.training/blogs/python-chatbot-project/
1. Import and load the data file

First, make a file name as train_chatbot.py. We import the necessary packages for our chatbot and initialize the variables we will use in our Python project.

The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [2]:
import json

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(r'C:\Users\mshch96\Desktop\Capstone\Github\ChatBot\intents.json').read()
intents = json.loads(data_file)

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

2. Preprocess data

When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. Tokenizing is the most basic and first thing you can do on text data. Tokenizing is the process of breaking the whole text into small parts like words.

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [5]:

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word 
        # ['Hi', 'there'] ['Is', 'anyone', 'there', '?']
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


Now we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [6]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

37 documents
8 classes ['dell_helpdesk', 'goodbye', 'greeting', 'internet_issue', 'options', 'password', 'software_crush', 'thanks']
77 unique lemmatized words ["'s", ',', 'a', 'access', 'admin', 'anyone', 'are', 'awesome', 'back', 'be', 'browser', 'bye', 'can', 'chatting', 'click', 'connection', 'could', 'crushed', 'day', 'dell', 'do', 'email', 'find', 'for', 'forget', 'froze', 'good', 'goodbye', 'hello', 'help', 'helpdesk', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'hr', 'i', 'if', 'is', 'later', 'me', 'microsoft', 'need', 'next', 'nice', 'no', 'not', 'number', 'offered', 'open', 'password', 'person', 'phone', 'provide', 'real', 'representitive', 'reset', 'response', 'see', 'server', 'support', 'team', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'web', 'what', 'without', 'you', 'youtube']


In [7]:
import pickle

In [8]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

3. Create training and testing data

Now, we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [9]:
import random
import numpy as np

In [10]:

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\mshch96\Anaconda2\envs\p36workshop\lib\site-packages\ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


4. Build the model

We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model. Let us save the model as ‘chatbot_model.h5’.

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [12]:

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save(r'C:\Users\mshch96\Desktop\Capstone\Github\ChatBot\chatbot_model.h5', hist)

print("model created")


Epoch 1/200
8/8 [==============================] - 0s 1ms/step - loss: 2.0419 - accuracy: 0.2420
Epoch 2/200
8/8 [==============================] - 0s 2ms/step - loss: 2.0198 - accuracy: 0.1508
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9791 - accuracy: 0.1951
Epoch 4/200
8/8 [==============================] - 0s 1ms/step - loss: 1.9965 - accuracy: 0.3391
Epoch 5/200
8/8 [==============================] - 0s 1ms/step - loss: 1.9472 - accuracy: 0.1651
Epoch 6/200
8/8 [==============================] - 0s 1ms/step - loss: 1.8861 - accuracy: 0.3519
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 1.8406 - accuracy: 0.2402
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 1.7457 - accuracy: 0.4233
Epoch 9/200
8/8 [==============================] - 0s 2ms/step - loss: 1.7159 - accuracy: 0.4320
Epoch 10/200
8/8 [==============================] - 0s 1ms/step - loss: 1.5375 - accuracy: 0.6418
Epoch 11/200
8/8 [===========

8/8 [==============================] - 0s 1ms/step - loss: 0.1616 - accuracy: 0.9586
Epoch 85/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0495 - accuracy: 0.9827
Epoch 86/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0982 - accuracy: 0.9698
Epoch 87/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0256 - accuracy: 1.0000
Epoch 88/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0460 - accuracy: 1.0000
Epoch 89/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0620 - accuracy: 0.9586
Epoch 90/200
8/8 [==============================] - 0s 1ms/step - loss: 0.1031 - accuracy: 1.0000
Epoch 91/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0251 - accuracy: 1.0000
Epoch 92/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0365 - accuracy: 1.0000
Epoch 93/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0593 - accuracy: 1.0000
Epoch 94/200
8/8 [===============

8/8 [==============================] - 0s 1ms/step - loss: 0.0379 - accuracy: 0.9827
Epoch 168/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0821 - accuracy: 0.9908
Epoch 169/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0185 - accuracy: 1.0000
Epoch 170/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0321 - accuracy: 1.0000
Epoch 171/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 172/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0483 - accuracy: 0.9771
Epoch 173/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0428 - accuracy: 1.0000
Epoch 174/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0887 - accuracy: 0.9586
Epoch 175/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 176/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0108 - accuracy: 1.0000
Epoch 177/200
8/8 [=====